In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import csv
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import seaborn as sns

from lieroy import se3
from recova.clustering import CenteredClusteringAlgorithm, RegistrationPairClusteringAdapter
from recova.covariance import SamplingCovarianceComputationAlgorithm, SamplingDistributionComputationAlgorithm
from recova.registration_result_database import RegistrationPairDatabase

In [3]:
db = RegistrationPairDatabase('/home/dlandry/dataset/db_kitti_02/')
pairs = db.registration_pairs()

In [5]:
pair = db.get_registration_pair('03', 702, 701)
pair.lie_matrix_of_results()

array([[ 9.34880376e-02,  2.36738101e-03,  2.13352055e-03,
        -8.46570474e-05,  5.59693319e-04,  1.50306441e-03],
       [ 6.26067877e-01,  5.70545811e-03,  6.69805007e-03,
        -2.01111630e-04, -9.68359745e-05,  1.09322788e-03],
       [ 1.81699425e-01,  2.33786204e-03,  2.46460759e-03,
        -1.25911742e-04,  3.97433207e-04,  1.10674463e-03],
       [ 6.02045178e-01,  5.54796727e-03,  6.99597690e-03,
        -2.35237909e-04,  5.87863105e-05,  1.15366466e-03],
       [ 6.01985455e-01,  5.38056251e-03,  7.00427871e-03,
        -2.17872730e-04,  6.47537527e-05,  1.18727959e-03],
       [ 6.02939427e-01,  5.24651725e-03,  6.91229012e-03,
        -2.33477404e-04,  5.93901786e-05,  1.22144609e-03],
       [ 6.02207839e-01,  5.26180910e-03,  7.09133968e-03,
        -1.84219607e-04,  7.85001321e-05,  1.20110181e-03],
       [ 6.01787806e-01,  5.37253870e-03,  7.03575741e-03,
        -2.24252115e-04,  7.29819149e-05,  1.24853349e-03],
       [ 6.01923645e-01,  5.31342113e-03,  6.861

In [ ]:
clustering_algo = CenteredClusteringAlgorithm(radius=0.005, k=20, n_seed_init=20)
clustering_algo.seed_selector = 'localized'
clustering = RegistrationPairClusteringAdapter(clustering_algo)

distribution_algo = SamplingDistributionComputationAlgorithm(clustering)
covariance_algo = SamplingCovarianceComputationAlgorithm(clustering_algorithm=clustering)

In [ ]:
def distance_of_one_pair(pair):
    print(pair)
    distribution = distribution_algo.compute(pair)
    
    mean, covariance = np.array(distribution['mean']), np.array(distribution['covariance'])
    
    ground_truth = pair.ground_truth()
    delta = se3.log(np.linalg.inv(ground_truth) @ mean)
    distance = np.linalg.norm(delta)

    return distance, 0.0

In [ ]:
distance_of_one_pair(pairs[5])

In [ ]:
#results = [distance_of_one_pair(x) for x in pairs]

with multiprocessing.Pool(processes=6) as pool:
    results = pool.map(distance_of_one_pair, pairs)
    

In [ ]:
covariances = [covariance_algo.compute(x) for x in pairs]

In [ ]:
cov_norms = [np.linalg.norm(x) for x in covariances]

In [ ]:
distances, _ = zip(*results)

In [ ]:
with open('/home/dlandry/distances_of_mean.csv', 'w') as f:
    writer = csv.DictWriter(f, ['location', 'reading', 'reference', 'distance', 'n_samples'])
    writer.writeheader()
    for i, pair in enumerate(pairs):
        writer.writerow({
            'location': pair.dataset,
            'reading': pair.reading,
            'reference': pair.reference,
            'distance': distances[i],
            'n_samples': 
        })

In [ ]:
sns.distplot(distances, rug=True)
plt.xlim([0.0, 0.2])
plt.show()

In [ ]:
plt.scatter(distances, cov_norms)
plt.ylim([0.0, 0.5])
plt.show()

In [ ]:
distances, closest = zip(*results)

In [ ]:
distance, closest = np.array(distances), np.array(closest)

In [ ]:
distance - closest

In [ ]:
np.max(distance - closest)

In [ ]:
distance - closest

In [ ]:
sns.distplot(closest, rug=True)
plt.xlim([0.0, 0.3])
plt.show()